# Installation and Basic Operations Notes for Kubernetes (the OS of the cloud) - tested on WSL2

- https://kubernetes.io/docs/home/
- https://www.docker.com/products/docker-desktop
- https://minikube.sigs.k8s.io/docs/tutorials/
- https://minikube.sigs.k8s.io/docs/handbook/accessing/

## Table of contents

- [Introduction to Kubernetes](#toc00)
- [Kubernetes as a Managed Service](#toc01)
- [Getting Kubernetes](#toc02)
- [Creating a Containerized App](#toc03)
- [Deploying the app to Kubernetes](#toc04)
- [Clean-up](#toc05)
- [Self-healing (Deployment Objects)](#toc06)
- [Scaling the app](#toc07)
- [Zero-downtime rolling update](#toc08)

---
<a id='toc00'></a>

## Introduction to Kubernetes
**Kubernetes (K8s)** (K8s -> "kates") is an orchestrator of cloud-native microservices applications. The name **Kubernetes** comes from the Greek word meaning `helmsman`, a nautical/sailing term for the person that steers a ship. The wheel of a ship is also called the `helm`. **Microservices** application is made of lots of small, specialised parts that are losely coupled to create a usefyl application. **Cloud-native** apps can self-heal, automatically scale, and be updated without downtime. An **orchestrator** brings together a set of microservices and organises them into an application that brings value, provides and manages cloud-native features such as scaling, self-healing, and updates. 

**Kubernetes** does an amazing job of abstracting underlying cloud and server infrastructure, so you don't have to care what cloud or servers your apps are running on. **Kuberenetes** is called `the operating system of the cloud`. Applications written for Kubernetes will run anywhere that you have Kubernetes. K8s apps can:
- be deployed to one cloud today and be re-deployed to another tomorrow,
- run multi-cloud,
- be more easily ramp onto a cloud and then ramp off back to on-prem.

We call a Kubernetes installation (one or more machines installed with Kubernetes) a `Kubernetes cluster`. The machines can be physical servers, virtual machines, cloud instances, a laptop, Raspberry Pi, and more. We refer to machines in K8s cluster as `nodes`. There are two types of nodes: master and worker. `Masters` (master nodes) host the `control plane` (brains of the cluster) and (worker) `Nodes` are where you run user applications.

**Masters** run many services that form the control pane but the `API Server` is the only part of Kubernetes cluster you directly interact with. **Nodes** run Linux or Windows user applications. The `Scheduler` chooses which Nodes to run user applications on. The `Store` is where the state of the cluster and all applications is stored. The `Cloud controller` allows K8s to integrate with cloud services such as storage and load-balancers.

All Nodes run a couple of services including `Kubelet` (main Kubernetes agent) and `Container runtime` (starts and stops containers). Most of the day-to-day management of K8s cluster is done using the Kubernetes command line toll called **kubectl** ("kube see tee ell").

Although Kubernetes orchestrates and runs containers, these containers must be wrapped in a Kubernetes construct called a `Pod`. Kubernetes uses `labels` to associate objects. Pods carry labels, whereas Service objects use them to make selections.

**Kubernetes** came into existence in the summer of 2014 as an open-source next gext generation of Borg and Omega (Google's internal tools) for managing containers.

---
<a id='toc01'></a>

## Kubernetes as a Managed Service

- [Linode](https://www.linode.com/)
- [AWS Elastic Kubernetes Service (EKS)]()
- [Azure Kubernetes Service (AKS)]()
- [Google Kubernetes Engine (GKE)]()

---
<a id='toc02'></a>

## Getting Kubernetes

### Integrated with Docker Desktop
With [Docker Desktop](https://www.docker.com/products/docker-desktop) you get Docker runtime and a `single-node` (single node acts as a Master and a worker Node) Kubernetes cluster that is great for development and learning, and kubectl command line utility. On Mac OS your single-node K8s cluster will run inside a lightweight VM, on WIndows it can run natively on WSL2 (Windows Subsystem for Linux, ver. 2). Enable Kubernetes in Docker Desktop preferences.
```
# Verify Docker and K8s installation
docker --version
kubectl version -o yaml
```

### Minikube
[minikube](https://minikube.sigs.k8s.io/docs/start/) quickly sets up a local Kubernetes cluster on macOS, Linux, and Windows.
```
# Choose the rigth version from: https://minikube.sigs.k8s.io/docs/start/
#curl -LO https://storage.googleapis.com/minikube/releases/v1.16.0/minikube-linux-amd64
curl -LO https://storage.googleapis.com/minikube/releases/latest/minikube-linux-amd64
sudo install minikube-linux-amd64 /usr/local/bin/minikube

# Verify Docker and K8s installation
docker --version
kubectl version -o yaml

# Test
minikube status
minikube start
#minikube start --nodes 2
kubectl get po -A

minikube dashboard

kubectl create deployment hello-minikube --image=k8s.gcr.io/echoserver:1.4
kubectl expose deployment hello-minikube --type=NodePort --port=8080

kubectl get services hello-minikube

# The easiest way to access this service is to let minikube launch a web browser for you
minikube service hello-minikube

# Your application is now available at http://localhost:7080/.
kubectl port-forward service/hello-minikube 7080:8080

minikube stop
minikube delete --all
```

### Kubectl
You may already have kubectl installed via another tool such as Docker Desktop or minikube.
```
# Mac OS
kubectl version --client -o yaml
brew install kubectl

# kubeconfig file is located (in order to connect to a cluster, take kubeconfig file from the provider and copy it here
~/.kube/config

kubectl get nodes
```

## git
```
# Mac OS
brew install git
git --version
```

---
<a id='toc03'></a>

## Creating a Containerized App
The typical `workflow for deploying apps on Kubernetes`:
1. Write the app code
2. Containerize the app (using Docker)
3. Share the app image on registry (Docker Hub)
4. Run the app on / deploy it to K8s (minikube)

### Prerequisites
- the `git` command line tool
- Docker (Desktop)
- [Docker Hub account (Docker ID)](https://hub.docker.com/)

### 1. Write the Application Code
```
# The original files come from
git clone https://github.com/nigelpoulton/qsk-book

# They are copied ti the k8s_app sobfolder of the notebooks folder in this repo
cd k8s_app/App
```

### 2. Containerize the App (Docker)
```
# Build the container image (==containerized app)
cd ~/git/MLOps/notebooks/k8s_app/App
docker image build -t ksatola/k8s_sample_app_img:1.0 .

# Confirm the image existis locally
docker image ls
```

### 3. Host the app image on a registry (Docker Hub)
```
# Authentication may be required
docker login

# Push the image to the registry
docker image push ksatola/k8s_sample_app_img:1.0

# Check in https://hub.docker.com/
```

---
<a id='toc04'></a>

## Deploying the app to Kubernetes

### Start your Kubernetes cluster
```
minikube start
#minikube start --nodes 2
cat ~/.kube/config

# List all contexts defined in your kubeconfig file
kubectl config get-contexts

# Switch to another context
kubectl config use-context <NAME>
```
### Verify your Kubernetes cluster
```
kubectl get nodes

# List cluster Pods running
kubectl get pods
```
### Deploy the app to Kubernetes
```
# The app will be deployed in a Pod called first-pod and defined in a YAML file pod.yml
cd ~/git/MLOps/notebooks/k8s_app
kubectl apply -f pod.yml

kubectl get pods
kubectl describe pod first-pod
```
### Connect to the app
Connecting to the app requires a separate object (something running on Kubernetes) called a `Service`.
```
# Deploy a Service object to provide connectivity to the app running in the Pod
# The svc-local.yml file defines a Service object to provide connectivity on a local cluster
cd ~/git/MLOps/notebooks/k8s_app
kubectl apply -f svc-local.yml

# Get services
kubectl get svc

# https://minikube.sigs.k8s.io/docs/handbook/accessing/
minikube service --url svc-local
```

---
<a id='toc05'></a>

## Clean-up
```
kubectl delete svc svc-local
kubectl delete pod first-pod
```

---
<a id='toc06'></a>

## Self-healing (Deployment Objects)
The `Deployment` Kubernetes object can be used to provide self-healing, scaling and rolling updates. As with `Pods` and `Service` objects, `Deployments` are defined declaratively in YAML manifest files. There are two elements to the working of a Deployment:
1. The Deployment object (YAML configuration that defines an application)
1. The Deployment controller (control plane process that is constantly monitoring the cluster making sure all Deployment objects are implemented and running -> the `Observed state` matches `Desired state` == `reconciliation` process)

The terms `Pod`, `instance`, and `replica` are used to mean the same thing.

### Self-healing from a Pod failure
```
# Start an empty cluster
minikube start --nodes 2
kubectl get pods
kubectl get deployments

# Deploy the Deployment to the cluster
cd ~/git/MLOps/notebooks/k8s_app
kubectl apply -f deploy.yml

# Check the status of the Deployment
kubectl get deployments
kubectl get pods

# Simulate Pod failure
kubectl delete pod k8s-app-deploy-f486796f7-5m48t

minikube delete

```
### Self-healing from a Node failure
When a `Node` fails, any `Pods` running on it are lost. The `Deployment` controller will start `Pods` replacements on a different `Node`. `Node pools` mechanism offered by some cloud K8s service providers can re-establish a Node, but this is not K8s feature. In the cloud scenario, once the failed Node is restored, K8s will not re-balance Pods.
```
# You must have multinode cluster to follow this scenario
# Start an empty cluster
minikube start --nodes 5
kubectl get nodes

# Deploy the Deployment to the cluster
cd ~/git/MLOps/notebooks/k8s_app
kubectl apply -f deploy.yml
kubectl get deployments
kubectl get pods -o wide

# Simulate Node failure
# Not working for mini
kubectl delete node minikube-m04

minikube delete
```

---
<a id='toc07'></a>

## Scaling the app
Kubernetes has a separate object, called a `Horizontal Pod Autoscaler (HPA)` for automatic scaling, however the examples below use manual scale up and down.
```
# Start an empty cluster
minikube start --nodes 3

# Deploy the Deployment to the cluster
cd ~/git/MLOps/notebooks/k8s_app
kubectl apply -f deploy.yml

kubectl get deployments

# --------
# Scale up (declaratively)
kubectl get deployment k8s-app-deploy

# Manually edit the Deployment YAML file, 
# increasing the number of replicas to 10, 
# and resend it to Kubernetes
kubectl apply -f deploy.yml

kubectl get deployment k8s-app-deploy
kubectl get pods

# New pods are mostly created in a new Node (if there are more than 1)
kubectl get pods -o wide

# --------
# Scale down (imperatively)
kubectl scale --replicas 5 deployment/k8s-app-deploy
kubectl get pods -o wide

```

---
<a id='toc08'></a>

## Zero-downtime rolling update
The `rolling update` forces Kubernetes to update one replica at a time in a methodical manner until all 5 replicas are running the new version.
```
# Start an empty cluster
minikube start --nodes 3

# Deploy the Deployments to the cluster (App and Service)
cd ~/git/MLOps/notebooks/k8s_app
kubectl apply -f deploy.yml -f svc-local.yml

kubectl get deployments
kubectl get svc

minikube service --url svc-local

# --------
# Rolling update

# Prepare a new app version and containaraze it
cd ~/git/MLOps/notebooks/k8s_app/App1-1
docker image build -t ksatola/k8s_sample_app_img:1.1 .
docker image push ksatola/k8s_sample_app_img:1.1

# Extend deploy.yml with RollingUpdate strategy (see rolling-update.yml) - lines 10-15 and 26

kubectl apply -f rolling-update.yml

# Monitor
kubectl rollout status deployment k8s-app-deploy

# And via web-browser

# Clean-up
kubectl get deployments
kubectl delete deployment k8s-app-deploy

kubectl get svc
kubectl delete svc svc-local

```

## Read:
```
https://kubernetes.io/docs/tasks/access-application-cluster/ingress-minikube/

# Enable the Ingress controller
minikube addons enable ingress

# Verify that the NGINX Ingress controller is running
kubectl get pods -n ingress-nginx

# Create a Deployment
kubectl create deployment web --image=gcr.io/google-samples/hello-app:1.0

# Expose the Deployment
kubectl expose deployment k8s-app-deploy --type=NodePort --port=2345

# Verify the Service is created and is available on a node port
kubectl get service k8s-app-deploy

# Get the service URL
minikube service svc-local --url

```

## Read: https://kubernetes.io/blog/2020/05/21/wsl-docker-kubernetes-on-the-windows-desktop/

## Enable the Dashboard service
```
# https://kubernetes.io/blog/2020/05/21/wsl-docker-kubernetes-on-the-windows-desktop/
minikube dashboard



kubectl cluster-info
# Kubernetes control plane is running at https://127.0.0.1:64907

# https://kubernetes.io/blog/2020/05/21/wsl-docker-kubernetes-on-the-windows-desktop/
# Install the Dashboard application into our cluster
kubectl apply -f https://raw.githubusercontent.com/kubernetes/dashboard/v2.0.0-rc6/aio/deploy/recommended.yaml
# Check the resources it created based on the new namespace created
kubectl get all -n kubernetes-dashboard

# Create a new ServiceAccount
kubectl apply -f - <<EOF
apiVersion: v1
kind: ServiceAccount
metadata:
  name: admin-user
  namespace: kubernetes-dashboard
EOF
# Create a ClusterRoleBinding for the ServiceAccount
kubectl apply -f - <<EOF
apiVersion: rbac.authorization.k8s.io/v1
kind: ClusterRoleBinding
metadata:
  name: admin-user
roleRef:
  apiGroup: rbac.authorization.k8s.io
  kind: ClusterRole
  name: cluster-admin
subjects:
- kind: ServiceAccount
  name: admin-user
  namespace: kubernetes-dashboard
EOF

# Get the Token for the ServiceAccount
kubectl -n kubernetes-dashboard describe secret $(kubectl -n kubernetes-dashboard get secret | grep admin-user | awk '{print $1}')
# Copy the token and copy it into the Dashboard login and press "Sign in"

eyJhbGciOiJSUzI1NiIsImtpZCI6IjNmOWtZa2ZPbnl6ZnlkaENmVlE0RDZtMk5SZnhsRHQ4VmxwRnFUMmlQLU0ifQ.eyJpc3MiOiJrdWJlcm5ldGVzL3NlcnZpY2VhY2NvdW50Iiwia3ViZXJuZXRlcy5pby9zZXJ2aWNlYWNjb3VudC9uYW1lc3BhY2UiOiJrdWJlcm5ldGVzLWRhc2hib2FyZCIsImt1YmVybmV0ZXMuaW8vc2VydmljZWFjY291bnQvc2VjcmV0Lm5hbWUiOiJhZG1pbi11c2VyLXRva2VuLXo0dm5tIiwia3ViZXJuZXRlcy5pby9zZXJ2aWNlYWNjb3VudC9zZXJ2aWNlLWFjY291bnQubmFtZSI6ImFkbWluLXVzZXIiLCJrdWJlcm5ldGVzLmlvL3NlcnZpY2VhY2NvdW50L3NlcnZpY2UtYWNjb3VudC51aWQiOiJiMzA4NWNlYi00YTEyLTQ3NzItOWMxOS0yNjQxYTA4NDMxZTgiLCJzdWIiOiJzeXN0ZW06c2VydmljZWFjY291bnQ6a3ViZXJuZXRlcy1kYXNoYm9hcmQ6YWRtaW4tdXNlciJ9.pIkBd0eNS4xLAr0-eZcykBCxmwXeSbmWl-rYqfxmWLVc-Up_bf51Q0VP_xmpqqWQPwlmmH41AjDZmz_Mop23HAlwGhX9ZKKjDhjEweOf-dRUE3Qo8lWPi_6kaGMYD6trhapu8aUuweHV5j6XNnP6E0yad2oI74--u8AsyhEoYTPbMXYNPZl6HKCLTA72NmhWMCW3ZnJ7xRPjTSJafdBRLEs51C3cmYX7j3vDLEWuYK59VHr_Bq-yPej4OPAGzqyFTDN-uCb02rq-lxUteBRTrp6oDS5wvfKc1DOKnr29Ua4XoVMDNdKDuYGDY8pzQkKWYrhp-D8ebaO_wFK9DtsWWw

# Start a kubectl proxy
kubectl proxy
# Enter the URL on your browser: http://localhost:8001/api/v1/namespaces/kubernetes-dashboard/services/https:kubernetes-dashboard:/proxy/
```